<a href="https://colab.research.google.com/github/AnthonyAugust/Projetos-Finalizados/blob/Projetos-Finalizados/Reconhecimento_Facial_(Perfeito).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Bibliotecas e Imports
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io

In [2]:
# =========================
# FUNÇÕES AUXILIARES
# =========================

def js_to_image(js_reply):
    """Converte objeto JS em imagem OpenCV"""
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def bbox_to_bytes(bbox_array):
    """Converte overlay de bounding box em bytes para JS"""
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='PNG')
    bbox_bytes = 'data:image/png;base64,{}'.format(
        str(b64encode(iobuf.getvalue()), 'utf-8')
    )
    return bbox_bytes

In [3]:

# Inicializa Haar Cascade
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

In [4]:
# =========================
# STREAM DE VÍDEO NO COLAB
# =========================

def video_stream():
    js = Javascript('''
    var video, div=null, stream, captureCanvas, imgElement, labelElement;
    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
        stream.getVideoTracks()[0].stop();
        video.remove(); div.remove();
        video = div = stream = imgElement = captureCanvas = labelElement = null;
    }

    function onAnimationFrame() {
        if (!shutdown) { window.requestAnimationFrame(onAnimationFrame); }
        if (pendingResolve) {
            var result = "";
            if (!shutdown) {
                captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                result = captureCanvas.toDataURL('image/jpeg', 0.8)
            }
            var lp = pendingResolve; pendingResolve = null; lp(result);
        }
    }

    async function createDom() {
        if (div !== null) { return stream; }

        div = document.createElement('div');
        div.style.border = '2px solid black';
        div.style.padding = '3px';
        div.style.width = '100%'; div.style.maxWidth = '600px';
        document.body.appendChild(div);

        const modelOut = document.createElement('div');
        modelOut.innerHTML = "<span>Status:</span>";
        labelElement = document.createElement('span');
        labelElement.innerText = 'Capturando Imagem';
        labelElement.style.fontWeight = 'bold';
        modelOut.appendChild(labelElement);
        div.appendChild(modelOut);

        video = document.createElement('video');
        video.style.display = 'block';
        video.width = div.clientWidth - 6;
        video.setAttribute('playsinline', '');
        video.onclick = () => { shutdown = true; };
        stream = await navigator.mediaDevices.getUserMedia({video: true});
        div.appendChild(video);

        imgElement = document.createElement('img');
        imgElement.style.position = 'absolute';
        imgElement.style.zIndex = 1;
        imgElement.onclick = () => { shutdown = true; };
        div.appendChild(imgElement);

        const instruction = document.createElement('div');
        instruction.innerHTML = '<span style="color:red;font-weight:bold;">Clique no vídeo para encerrar</span>';
        div.appendChild(instruction);
        instruction.onclick = () => { shutdown = true; };

        video.srcObject = stream;
        await video.play();

        captureCanvas = document.createElement('canvas');
        captureCanvas.width = 640; captureCanvas.height = 480;

        window.requestAnimationFrame(onAnimationFrame);
        return stream;
    }

    async function stream_frame(label, imgData) {
        if (shutdown) { removeDom(); shutdown=false; return ''; }
        await createDom();
        if (label != "") { labelElement.innerHTML = label; }
        if (imgData != "") {
            var videoRect = video.getClientRects()[0];
            imgElement.style.top = videoRect.top + "px";
            imgElement.style.left = videoRect.left + "px";
            imgElement.style.width = videoRect.width + "px";
            imgElement.style.height = videoRect.height + "px";
            imgElement.src = imgData;
        }
        var result = await new Promise(function(resolve){ pendingResolve = resolve; });
        shutdown = false;
        return {'img': result};
    }
    ''')
    display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

In [6]:
# =========================
# LOOP PRINCIPAL
# =========================

video_stream()
label_html = 'Capturando Imagem'
bbox = ''

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        print("Captura encerrada pelo usuário.")
        break

    # converte JS -> OpenCV
    img = js_to_image(js_reply["img"])

    # overlay transparente
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # conversão para grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # detecção de faces
    faces = face_cascade.detectMultiScale(gray)
    for (x,y,w,h) in faces:
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    # alpha do overlay
    bbox_array[:,:,3] = (bbox_array.max(axis=2)>0).astype(int) * 255

    # converter overlay em bytes para JS
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

/tmp/ipython-input-4104504733.py:14: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')


Captura encerrada pelo usuário.
